# Validation Predictions Generator

## Environment Setup

### Imports

In [1]:
# System
import os

# Data / MONAI
import pandas as pd
from monai.data import DataLoader

# Torch
import torch
from torch.utils.data import SequentialSampler

# Utils
from Transforms import Transforms
from Datasets import UCSF_Dataset
from Models import SEGRESNET, UNET, AHNET, UNTR
from Inference import gen_predictions

### Config

In [2]:
# Check if CUDA is available
device = None
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on GPU")
else:
    device = torch.device("cpu")
    print("Running on CPU")

# Print the device
print(f"Device: {device}")

Running on GPU
Device: cuda


In [3]:
seed = 33
pd.set_option("display.max_columns", None)

## Load

### Models

In [4]:
models = {}
for model in ['AHNet', 'SegResNet', 'UNet', 'UNETR']:
	for files in os.listdir(f'../outputs/{model}'):
		if files.endswith('.pth'):
			models[model] = f'../outputs/{model}/{files}'

models

{'AHNet': '../outputs/AHNet/best_AHNet_92.pth',
 'SegResNet': '../outputs/SegResNet/best_SegResNet_96.pth',
 'UNet': '../outputs/UNet/best_UNet_97.pth',
 'UNETR': '../outputs/UNETR/best_UNETR_99.pth'}

### Train Data

In [5]:
# Load Subjects Information
train_df = pd.read_csv('../data/TRAIN.csv')
train_df['BraTS-seg'] = train_df['BraTS-seg'].apply(lambda x: f'../{x}')
train_df['T1post'] = train_df['T1post'].apply(lambda x: f'../{x}')
train_df['T1pre'] = train_df['T1pre'].apply(lambda x: f'../{x}')
train_df['FLAIR'] = train_df['FLAIR'].apply(lambda x: f'../{x}')
train_df['T2Synth'] = train_df['T2Synth'].apply(lambda x: f'../{x}')

# Parameters
b_size = 1
t_size = None

# Transforms
transforms = Transforms(seed)

# Train Dataset
train_images = [train_df['T1pre'], train_df['FLAIR'], train_df['T1post'], train_df['T2Synth']]
train_labels = train_df['BraTS-seg']

train_dataset = UCSF_Dataset(train_images, train_labels, transforms.val(), t_size)

# Samplers
train_sampler = SequentialSampler(train_dataset)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=b_size, shuffle=False, sampler=train_sampler)

train_df.head()

,SubjectID,Sex,CancerType,ScannerType,In-plane voxel size (mm),Matrix size,Prior Craniotomy/Biopsy/Resection,Age,Scanner Strength (Tesla),Slice Thickness (mm),NumberMetastases,VolumeMetastases_mm3,S-NM,S-V,S-VMax,S-VMin,S-VMean,S-VStd,S-VDiff,T1pre,FLAIR,T1post,T2Synth,Seg,BraTS-seg
0,100381A,Male,Lung,GE 1.5 T Signa HDxt,0.86x0.86,256x256x126,No,71.0,1.5,1.5,4.0,3082.045582,4.0,3082.045541,2445.030377,77.549672,770.511385,974.806455,4.146686e-05,../data/raw/UCSF_BrainMetastases_TRAIN/100381A...,../data/raw/UCSF_BrainMetastases_TRAIN/100381A...,../data/raw/UCSF_BrainMetastases_TRAIN/100381A...,../data/raw/UCSF_BrainMetastases_TRAIN/100381A...,data/raw/UCSF_BrainMetastases_TRAIN/100381A/10...,../data/raw/UCSF_BrainMetastases_TRAIN/100381A...
1,100414B,Female,Breast,GE 1.5 T Signa HDxt,0.59x0.59,512x512x50,No,52.0,1.5,3.0,5.0,84.446590,5.0,84.446586,22.656401,11.328201,16.889317,3.940386,3.629621e-06,../data/raw/UCSF_BrainMetastases_TRAIN/100414B...,../data/raw/UCSF_BrainMetastases_TRAIN/100414B...,../data/raw/UCSF_BrainMetastases_TRAIN/100414B...,../data/raw/UCSF_BrainMetastases_TRAIN/100414B...,data/raw/UCSF_BrainMetastases_TRAIN/100414B/10...,../data/raw/UCSF_BrainMetastases_TRAIN/100414B...
2,100132B,Male,Lung,GE 1.5 T Signa HDxt,0.5x0.5,512x512x156,No,55.0,1.5,1.2,5.0,734.400029,5.0,734.400029,236.700009,81.300003,146.880006,64.989060,1.136868e-13,../data/raw/UCSF_BrainMetastases_TRAIN/100132B...,../data/raw/UCSF_BrainMetastases_TRAIN/100132B...,../data/raw/UCSF_BrainMetastases_TRAIN/100132B...,../data/raw/UCSF_BrainMetastases_TRAIN/100132B...,data/raw/UCSF_BrainMetastases_TRAIN/100132B/10...,../data/raw/UCSF_BrainMetastases_TRAIN/100132B...
3,100212A,Female,Lung,GE 1.5 T Signa HDxt,1.17x1.17,256x256x98,No,52.0,1.5,1.5,9.0,208.062478,9.0,208.062474,43.260514,16.480196,23.118053,7.876000,3.481233e-06,../data/raw/UCSF_BrainMetastases_TRAIN/100212A...,../data/raw/UCSF_BrainMetastases_TRAIN/100212A...,../data/raw/UCSF_BrainMetastases_TRAIN/100212A...,../data/raw/UCSF_BrainMetastases_TRAIN/100212A...,data/raw/UCSF_BrainMetastases_TRAIN/100212A/10...,../data/raw/UCSF_BrainMetastases_TRAIN/100212A...
4,100243B,Female,Breast,GE 1.5 T Signa HDxt,0.86x0.86,256x256x100,No,55.0,1.5,1.5,6.0,517.367105,6.0,517.367098,173.932836,8.862820,86.227850,52.711028,6.960829e-06,../data/raw/UCSF_BrainMetastases_TRAIN/100243B...,../data/raw/UCSF_BrainMetastases_TRAIN/100243B...,../data/raw/UCSF_BrainMetastases_TRAIN/100243B...,../data/raw/UCSF_BrainMetastases_TRAIN/100243B...,data/raw/UCSF_BrainMetastases_TRAIN/100243B/10...,../data/raw/UCSF_BrainMetastases_TRAIN/100243B...


### Val Data

In [6]:
# Load Subjects Information
val_df = pd.read_csv('../data/VAL.csv')
val_df['BraTS-seg'] = val_df['BraTS-seg'].apply(lambda x: f'../{x}')
val_df['T1post'] = val_df['T1post'].apply(lambda x: f'../{x}')
val_df['T1pre'] = val_df['T1pre'].apply(lambda x: f'../{x}')
val_df['FLAIR'] = val_df['FLAIR'].apply(lambda x: f'../{x}')
val_df['T2Synth'] = val_df['T2Synth'].apply(lambda x: f'../{x}')

# Parameters
b_size = 1
t_size = None

# Transforms
transforms = Transforms(seed)

# Val Dataset
val_images = [val_df['T1pre'], val_df['FLAIR'], val_df['T1post'], val_df['T2Synth']]
val_labels = val_df['BraTS-seg']

val_dataset = UCSF_Dataset(val_images, val_labels, transforms.val(), t_size)

# Samplers
val_sampler = SequentialSampler(val_dataset)

# DataLoaders
val_loader = DataLoader(val_dataset, batch_size=b_size, shuffle=False, sampler=val_sampler)

val_df.head()

,SubjectID,Sex,CancerType,ScannerType,In-plane voxel size (mm),Matrix size,Prior Craniotomy/Biopsy/Resection,Age,Scanner Strength (Tesla),Slice Thickness (mm),NumberMetastases,VolumeMetastases_mm3,S-NM,S-V,S-VMax,S-VMin,S-VMean,S-VStd,S-VDiff,T1pre,FLAIR,T1post,T2Synth,Seg,BraTS-seg
0,100237A,Female,Breast,Philips 1.5 T Achieva,0.71x0.71,336x336x110,No,72.0,1.5,1.5,2.0,3462.245063,2.0,3462.244865,3012.244869,449.999996,1731.122432,1281.122437,1.981112e-04,../data/raw/UCSF_BrainMetastases_TRAIN/100237A...,../data/raw/UCSF_BrainMetastases_TRAIN/100237A...,../data/raw/UCSF_BrainMetastases_TRAIN/100237A...,../data/raw/UCSF_BrainMetastases_TRAIN/100237A...,data/raw/UCSF_BrainMetastases_TRAIN/100237A/10...,../data/raw/UCSF_BrainMetastases_TRAIN/100237A...
1,100219A,Female,Lung,GE 3.0 T Discovery MR750,0.43x0.43,512x512x33,No,83.0,3.0,5.0,1.0,90.474619,1.0,90.474617,90.474617,90.474617,90.474617,0.000000,1.217276e-06,../data/raw/UCSF_BrainMetastases_TRAIN/100219A...,../data/raw/UCSF_BrainMetastases_TRAIN/100219A...,../data/raw/UCSF_BrainMetastases_TRAIN/100219A...,../data/raw/UCSF_BrainMetastases_TRAIN/100219A...,data/raw/UCSF_BrainMetastases_TRAIN/100219A/10...,../data/raw/UCSF_BrainMetastases_TRAIN/100219A...
2,100363A,Female,Lung,GE 1.5 T Signa HDxt,0.86x0.86,256x256x102,No,69.0,1.5,1.5,5.0,167.285724,5.0,167.285721,65.363295,9.970672,33.457144,18.263108,2.250718e-06,../data/raw/UCSF_BrainMetastases_TRAIN/100363A...,../data/raw/UCSF_BrainMetastases_TRAIN/100363A...,../data/raw/UCSF_BrainMetastases_TRAIN/100363A...,../data/raw/UCSF_BrainMetastases_TRAIN/100363A...,data/raw/UCSF_BrainMetastases_TRAIN/100363A/10...,../data/raw/UCSF_BrainMetastases_TRAIN/100363A...
3,100354A,Female,Lung,Philips 1.5 T Achieva,0.69x0.69,320x320x104,No,53.0,1.5,1.5,6.0,385.687500,6.0,385.687500,289.265625,4.253906,64.281250,101.268739,0.000000e+00,../data/raw/UCSF_BrainMetastases_TRAIN/100354A...,../data/raw/UCSF_BrainMetastases_TRAIN/100354A...,../data/raw/UCSF_BrainMetastases_TRAIN/100354A...,../data/raw/UCSF_BrainMetastases_TRAIN/100354A...,data/raw/UCSF_BrainMetastases_TRAIN/100354A/10...,../data/raw/UCSF_BrainMetastases_TRAIN/100354A...
4,100303A,Male,Lung,GE 1.5 T Signa HDxt,0.86x0.86,256x256x108,No,62.0,1.5,1.5,6.0,58.716181,6.0,58.716180,23.264902,5.539262,9.786030,6.182064,7.899870e-07,../data/raw/UCSF_BrainMetastases_TRAIN/100303A...,../data/raw/UCSF_BrainMetastases_TRAIN/100303A...,../data/raw/UCSF_BrainMetastases_TRAIN/100303A...,../data/raw/UCSF_BrainMetastases_TRAIN/100303A...,data/raw/UCSF_BrainMetastases_TRAIN/100303A/10...,../data/raw/UCSF_BrainMetastases_TRAIN/100303A...


### Test Data

In [7]:
# Load Subjects Information
test_df = pd.read_csv('../data/TEST.csv')
test_df['BraTS-seg'] = test_df['BraTS-seg'].apply(lambda x: f'../{x}')
test_df['T1post'] = test_df['T1post'].apply(lambda x: f'../{x}')
test_df['T1pre'] = test_df['T1pre'].apply(lambda x: f'../{x}')
test_df['FLAIR'] = test_df['FLAIR'].apply(lambda x: f'../{x}')
test_df['T2Synth'] = test_df['T2Synth'].apply(lambda x: f'../{x}')

# Parameters
b_size = 1
t_size = None

# Transforms
transforms = Transforms(seed)

# Val Dataset
test_images = [test_df['T1pre'], test_df['FLAIR'], test_df['T1post'], test_df['T2Synth']]
test_labels = test_df['BraTS-seg']

test_dataset = UCSF_Dataset(test_images, test_labels, transforms.val(), t_size)

# Samplers
test_sampler = SequentialSampler(test_dataset)

# DataLoaders
test_loader = DataLoader(test_dataset, batch_size=b_size, shuffle=False, sampler=test_sampler)

test_df.head()

,SubjectID,Sex,CancerType,ScannerType,In-plane voxel size (mm),Matrix size,Prior Craniotomy/Biopsy/Resection,Age,Scanner Strength (Tesla),Slice Thickness (mm),NumberMetastases,VolumeMetastases_mm3,S-NM,S-V,S-VMax,S-VMin,S-VMean,S-VStd,S-VDiff,T1pre,FLAIR,T1post,T2Synth,Seg,BraTS-seg
0,100214B,Female,Melanoma,Philips 1.5 T Achieva,0.69x0.69,320x320x104,No,72.0,1.5,1.5,17.0,438.861328,17.0,438.861328,133.998047,4.253906,25.815372,37.943862,0.000000,../data/raw/UCSF_BrainMetastases_TRAIN/100214B...,../data/raw/UCSF_BrainMetastases_TRAIN/100214B...,../data/raw/UCSF_BrainMetastases_TRAIN/100214B...,../data/raw/UCSF_BrainMetastases_TRAIN/100214B...,data/raw/UCSF_BrainMetastases_TRAIN/100214B/10...,../data/raw/UCSF_BrainMetastases_TRAIN/100214B...
1,100340A,Female,Lung,GE 1.5 T Signa HDxt,1.17x1.17,256x256x98,No,64.0,1.5,1.5,9.0,1845.781983,9.0,1845.781952,716.888526,20.600245,205.086884,216.778603,0.000031,../data/raw/UCSF_BrainMetastases_TRAIN/100340A...,../data/raw/UCSF_BrainMetastases_TRAIN/100340A...,../data/raw/UCSF_BrainMetastases_TRAIN/100340A...,../data/raw/UCSF_BrainMetastases_TRAIN/100340A...,data/raw/UCSF_BrainMetastases_TRAIN/100340A/10...,../data/raw/UCSF_BrainMetastases_TRAIN/100340A...
2,100391A,Female,Lung,Philips 1.5 T Achieva,0.69x0.69,320x320x104,No,68.0,1.5,1.5,1.0,280.757812,1.0,280.757812,280.757812,280.757812,280.757812,0.000000,0.000000,../data/raw/UCSF_BrainMetastases_TRAIN/100391A...,../data/raw/UCSF_BrainMetastases_TRAIN/100391A...,../data/raw/UCSF_BrainMetastases_TRAIN/100391A...,../data/raw/UCSF_BrainMetastases_TRAIN/100391A...,data/raw/UCSF_BrainMetastases_TRAIN/100391A/10...,../data/raw/UCSF_BrainMetastases_TRAIN/100391A...
3,100190B,Female,Lung,GE 1.5 T Signa HDxt,0.86x0.86,256x256x104,No,42.0,1.5,1.5,3.0,163.962166,2.0,163.962164,132.942295,31.019869,81.981082,50.961213,0.000002,../data/raw/UCSF_BrainMetastases_TRAIN/100190B...,../data/raw/UCSF_BrainMetastases_TRAIN/100190B...,../data/raw/UCSF_BrainMetastases_TRAIN/100190B...,../data/raw/UCSF_BrainMetastases_TRAIN/100190B...,data/raw/UCSF_BrainMetastases_TRAIN/100190B/10...,../data/raw/UCSF_BrainMetastases_TRAIN/100190B...
4,100142A,Female,Neuroendocrine,GE 1.5 T Signa HDxt,0.86x0.86,256x256x106,No,64.0,1.5,1.5,2.0,1197.588524,2.0,1197.588508,1186.509984,11.078525,598.794254,587.715729,0.000016,../data/raw/UCSF_BrainMetastases_TRAIN/100142A...,../data/raw/UCSF_BrainMetastases_TRAIN/100142A...,../data/raw/UCSF_BrainMetastases_TRAIN/100142A...,../data/raw/UCSF_BrainMetastases_TRAIN/100142A...,data/raw/UCSF_BrainMetastases_TRAIN/100142A/10...,../data/raw/UCSF_BrainMetastases_TRAIN/100142A...


## Generate

In [8]:
spatial_size = (240, 240, 160)
ah_spatial_size = (256, 256, 160)

### Training

In [9]:
for model_name in models.keys():
    print(f"{model_name}")

    # Load Model
    model = None
    if model_name == 'SegResNet':
        model = SEGRESNET
    elif model_name == 'UNet':
        model = UNET
    elif model_name == 'AHNet':
        model = AHNET
    elif model_name == 'UNETR':
        model = UNTR
    else:
        print("Model not found")
        break
    model.to(device)
    model.load_state_dict(torch.load(models[model_name]))

    # Generate Predictions
    if model_name == 'AHNet':
        gen_predictions(model, model_name, train_loader, train_df, ah_spatial_size, 'train')
    else:
        gen_predictions(model, model_name, train_loader, train_df, spatial_size, 'train')
    

AHNet


100%|██████████| 248/248 [47:58<00:00, 11.61s/it]


SegResNet


100%|██████████| 248/248 [1:03:36<00:00, 15.39s/it]


UNet


100%|██████████| 248/248 [26:25<00:00,  6.39s/it]


UNETR


100%|██████████| 248/248 [1:06:06<00:00, 16.00s/it]


### Validation

In [10]:
for model_name in models.keys():
    print(f"{model_name}")

    # Load Model
    model = None
    if model_name == 'SegResNet':
        model = SEGRESNET
    elif model_name == 'UNet':
        model = UNET
    elif model_name == 'AHNet':
        model = AHNET
    elif model_name == 'UNETR':
        model = UNTR
    else:
        print("Model not found")
        break
    model.to(device)
    model.load_state_dict(torch.load(models[model_name]))

    # Generate Predictions
    if model_name == 'AHNet':
        gen_predictions(model, model_name, val_loader, val_df, ah_spatial_size, 'val')
    else:
        gen_predictions(model, model_name, val_loader, val_df, spatial_size, 'val')
    

AHNet


100%|██████████| 31/31 [13:18<00:00, 25.77s/it]


SegResNet


100%|██████████| 31/31 [14:40<00:00, 28.41s/it]


UNet


100%|██████████| 31/31 [08:37<00:00, 16.70s/it]


UNETR


100%|██████████| 31/31 [12:35<00:00, 24.37s/it]


### Test

In [11]:
for model_name in models.keys():
    print(f"{model_name}")

    # Load Model
    model = None
    if model_name == 'SegResNet':
        model = SEGRESNET
    elif model_name == 'UNet':
        model = UNET
    elif model_name == 'AHNet':
        model = AHNET
    elif model_name == 'UNETR':
        model = UNTR
    else:
        print("Model not found")
        break
    model.to(device)
    model.load_state_dict(torch.load(models[model_name]))

    # Generate Predictions
    if model_name == 'AHNet':
        gen_predictions(model, model_name, val_loader, val_df, ah_spatial_size, 'test')
    else:
        gen_predictions(model, model_name, val_loader, val_df, spatial_size, 'test')

AHNet


100%|██████████| 31/31 [08:32<00:00, 16.54s/it]


SegResNet


100%|██████████| 31/31 [10:02<00:00, 19.42s/it]


UNet


100%|██████████| 31/31 [03:37<00:00,  7.02s/it]


UNETR


100%|██████████| 31/31 [08:44<00:00, 16.93s/it]
